In [ ]:
#Importando a biblioteca que será usada
import pandas as pd

In [ ]:
#Importação dos dados das eleições municipais de 2020 no estado de SP. Site TSE
elc_sp = pd.read_csv('consulta_cand_2020_SP.csv',sep=';',encoding='latin1')

In [ ]:
#Filtragem para um df secundário, considerando só os candidatos que estavam disponíveis para receber votos
t = elc_sp[elc_sp['ST_CANDIDATO_INSERIDO_URNA']=='SIM']

In [ ]:
#Filtragem para considerarmos a prefeitos candidatos a reeleição
t = t[(t['DS_CARGO']=='PREFEITO')&(t['ST_REELEICAO']=='S')]

In [ ]:
#Análises exploratórias
t['DS_SITUACAO_CANDIDATURA'].unique()
t['DS_DETALHE_SITUACAO_CAND'].value_counts()
t['DS_SITUACAO_CANDIDATO_URNA'].value_counts()

In [ ]:
#Filtragem do df principal selecionando só as colunas que nos interessa
elc_sp = elc_sp[['CD_TIPO_ELEICAO','NR_TURNO','NM_UE','DS_CARGO','SQ_CANDIDATO','ST_CANDIDATO_INSERIDO_URNA','ST_REELEICAO']]

In [ ]:
#Filtragem para candidatos ao cargo de prefeito
elc_sp = elc_sp[elc_sp['DS_CARGO']=='PREFEITO']

In [ ]:
#Tamanho da amostra
elc_sp.groupby(['ST_REELEICAO','DS_SITUACAO_CANDIDATURA']).count()

In [ ]:
#Substituição de 'S' por 1 e 'N' por 0
elc_sp.loc[elc_sp['ST_REELEICAO'] == 'S','ST_REELEICAO'] = 1
elc_sp.loc[elc_sp['ST_REELEICAO'] == 'N','ST_REELEICAO'] = 0

In [ ]:
#Filtragem para considerarmos somente a eleição em primeiro turno
elc_sp = elc_sp[elc_sp['NR_TURNO']==1]

In [ ]:
#Filtragem para considerarmos somente eleições do tipo 2, ordinárias. Fonte: TSE
elc_sp = elc_sp[elc_sp['CD_TIPO_ELEICAO']==2]

In [ ]:
#Testes de validades dos filtros aplicados nos dados
elc_sp[(elc_sp['NR_TURNO']==2)|(elc_sp['CD_TIPO_ELEICAO']!=2)|(elc_sp['DS_CARGO']!='PREFEITO')]

In [ ]:
#Filtragem de candidatos inseridos na urna (possibilidade de receberem votos) e criando um novo df
D = elc_sp[elc_sp['ST_CANDIDATO_INSERIDO_URNA']=='SIM']

In [ ]:
#Agrupando por município e se é candidato a reeleição, contando o n e resetando o index 
D = elc_sp.groupby(['NM_UE','ST_REELEICAO']).count().reset_index()[['NM_UE','ST_REELEICAO']]

In [ ]:
#Agrupando por município 
D = D.groupby('NM_UE').sum().reset_index()

In [ ]:
#Conferindo a validade das manipulações, valores esperados: 0 e 1
D['ST_REELEICAO'].value_counts()

In [ ]:
#Exportando para csv um df com cada município de SP e a informação de que se havia ou não prefeito candidato a reeleição
D.to_csv('consulta_cand_reeleicao_2020_SP.csv',sep=';',encoding='latin1')